In [1]:
import os
import cv2
import shutil
import mediapy
import numpy as np
from multiprocessing import Pool

home_dir = ""

mean = 0
var = 0.3
sigma = var ** 0.5
new_dir = home_dir + f"_GSN{var}".replace(".", "")

if os.path.exists(new_dir):
    shutil.rmtree(new_dir)
os.makedirs(new_dir)

def randomize_gamma(imgs, gamma):
    lookup_table = np.array([((i / 255) ** (1.0 / gamma)) * 255 for i in np.arange(256)]).astype("uint8")
    return cv2.LUT(imgs, lookup_table)

def process_video(dirname, file):
    v_path = os.path.join(home_dir, dirname, file)
    new_v_path = os.path.join(new_dir, dirname, file)
    if v_path.endswith(".mp4"):
        video = mediapy.read_video(v_path)
        new_video = cv2.add(video, np.random.normal(mean, sigma, video.shape).astype('uint8'))
        new_video = randomize_gamma(new_video, np.random.uniform(0.5, 1.5))
        mediapy.write_video(new_v_path, new_video, fps=video.metadata.fps)
    elif v_path.endswith(".json"):
        shutil.copy(v_path, new_v_path)

def process_directory(dirname):
    if not os.path.isdir(os.path.join(home_dir, dirname)):
        shutil.copy(os.path.join(home_dir, dirname), os.path.join(new_dir, dirname))
        return
    new_subdir = os.path.join(new_dir, dirname)
    if not os.path.exists(new_subdir):
        os.mkdir(new_subdir)

    for file in sorted(os.listdir(os.path.join(home_dir, dirname))):
        process_video(dirname, file)

directories = sorted(os.listdir(home_dir))
pool = Pool(processes=16)
pool.map(process_directory, directories)
pool.close()
pool.join()
